In [0]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
meta = pd.read_csv("drive/My Drive/movies_metadata.csv", encoding='latin1')
imdb = pd.read_csv("drive/My Drive/MovieGenre.csv", encoding='latin1')
credits = pd.read_csv("drive/My Drive/credits.csv", encoding='latin1')
keywords = pd.read_csv("drive/My Drive/keywords.csv", encoding='latin1')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
print(meta.shape)
print(imdb.shape)
print(credits.shape)
print(keywords.shape)

(45466, 24)
(40108, 6)
(45476, 3)
(46419, 2)


In [0]:
meta['posterID'] = np.arange(meta.shape[0])
meta_cleaned = meta.copy()
meta_cleaned = meta_cleaned[meta_cleaned['imdb_id']!='0']
meta_cleaned['imdb_id'] = meta_cleaned['imdb_id'].str.replace('tt','')

In [0]:
meta_cleaned['imdb_id'] = pd.to_numeric(meta_cleaned['imdb_id'])
meta_cleaned = meta_cleaned[~np.isnan(meta_cleaned['imdb_id'])]
meta_cleaned['imdb_id'] = meta_cleaned['imdb_id'].astype(int)
meta_cleaned = meta_cleaned.drop_duplicates(subset=['imdb_id'], keep=False)

In [0]:
meta_cleaned.shape[0]

45387

In [0]:
imdb_cleaned = imdb.rename(columns={'imdbId': 'imdb_id'})
imdb_cleaned = imdb_cleaned.drop_duplicates(subset=['imdb_id'], keep=False)
imdb_cleaned.shape[0]

38922

In [0]:
credits_cleaned = credits.drop_duplicates(subset=['id'], keep=False)
keywords_cleaned = keywords.drop_duplicates(subset=['id'], keep=False)
print(credits_cleaned.shape[0])
print(keywords_cleaned.shape[0])

45389
44447


In [0]:
meta_cleaned['id'] = meta_cleaned['id'].astype(int)

In [0]:
combined = pd.merge(meta_cleaned, credits_cleaned, on='id')
combined = pd.merge(combined, keywords_cleaned, on='id')
combined = pd.merge(combined, imdb_cleaned, on='imdb_id')

In [0]:
combined.shape

(37504, 33)

In [0]:
combined['posterID'] = combined['posterID'].astype(str) + ".jpg"

In [0]:
all_data = combined.copy()

In [0]:
!unzip "/content/drive/My Drive/Colab Notebooks/poster.zip"

In [0]:
import os

n = meta.shape[0]

failed = []
for i in range(n):
    if not os.path.exists("poster/" + str(i) + ".jpg"):
        failed.append(i)
        
indices = list(map(lambda x: x not in failed, all_data['posterID'].str.replace('.jpg','').astype(int)))
all_data = all_data[indices]
print(all_data.shape)

(37210, 33)


In [0]:
all_data = all_data.reset_index(drop=True)

In [0]:
import cv2

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")
num_faces = np.zeros(all_data.shape[0])

brightness = np.zeros(all_data.shape[0])
saturation = np.zeros(all_data.shape[0])
hue = np.zeros(all_data.shape[0])
brightness_sd = np.zeros(all_data.shape[0])
saturation_sd = np.zeros(all_data.shape[0])
hue_sd = np.zeros(all_data.shape[0])

blue = np.zeros(all_data.shape[0])
blue_sd = np.zeros(all_data.shape[0])
green = np.zeros(all_data.shape[0])
green_sd = np.zeros(all_data.shape[0])
red = np.zeros(all_data.shape[0])
red_sd = np.zeros(all_data.shape[0])

for i in range(all_data.shape[0]):
  image = cv2.imread('poster/' + all_data['posterID'][i])
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = faceCascade.detectMultiScale(gray, minSize=(100, 100))
  num_faces[i] = len(faces)

  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  brightness[i] = np.mean(hsv[:,:,2])
  saturation[i] = np.mean(hsv[:,:,1])
  hue[i] = np.mean(hsv[:,:,0])
  brightness_sd[i] = np.std(hsv[:,:,2])
  saturation_sd[i] = np.std(hsv[:,:,1])
  hue_sd[i] = np.std(hsv[:,:,0])

  blue[i] = np.mean(image[:,:,0])
  green[i] = np.mean(image[:,:,1])
  red[i] = np.mean(image[:,:,2])
  blue_sd[i] = np.std(image[:,:,0])
  green_sd[i] = np.std(image[:,:,1])
  red_sd[i] = np.std(image[:,:,2])

  if i % 1000 == 0:
    print("iter", i)

iter 0
iter 1000
iter 2000
iter 3000
iter 4000
iter 5000
iter 6000
iter 7000
iter 8000
iter 9000
iter 10000
iter 11000
iter 12000
iter 13000
iter 14000
iter 15000
iter 16000
iter 17000
iter 18000
iter 19000
iter 20000
iter 21000
iter 22000
iter 23000
iter 24000
iter 25000
iter 26000
iter 27000
iter 28000
iter 29000
iter 30000
iter 31000
iter 32000
iter 33000
iter 34000
iter 35000
iter 36000
iter 37000


In [0]:
all_data['num_faces'] = num_faces

all_data['brightness'] = brightness
all_data['saturation'] = saturation
all_data['hue'] = hue
all_data['brightness_sd'] = brightness_sd
all_data['saturation_sd '] = saturation_sd
all_data['hue_sd'] = hue_sd

all_data['blue'] = blue
all_data['blue_sd'] = blue_sd
all_data['green'] = green
all_data['green_sd'] = green_sd
all_data['red'] = red
all_data['red_sd '] = red_sd

In [0]:
all_data.shape

(37210, 46)

In [0]:
all_data.to_csv("drive/My Drive/preprocessed.csv", index = False)

In [0]:
all_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,posterID,cast,crew,keywords,Imdb Link,Title,IMDB Score,Genre,Poster,num_faces,brightness,saturation,hue,brightness_sd,saturation_sd,hue_sd,blue,blue_sd,green,green_sd,red,red_sd
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,0.jpg,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...,0.0,184.775439,190.399400,82.805137,58.496526,80.933495,46.984021,147.046133,83.046441,127.812661,59.618615,79.571854,88.574055
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1.jpg,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...,0.0,166.807096,91.547633,65.296239,83.413251,67.319422,61.296055,132.472598,84.508776,137.041430,79.140584,137.619493,85.319415
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,2.jpg,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,0.0,139.380694,112.907880,48.908106,94.073346,70.979805,65.406756,97.464385,78.935691,106.743824,89.725921,136.728580,96.124722
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,3.jpg,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...,1.0,193.179470,78.009035,43.298337,71.637853,74.316934,57.431210,148.388970,88.371216,161